<a href="https://colab.research.google.com/github/staller84/minhee/blob/master/hw4_nn_pmh_with_colab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
from skimage.util.shape import view_as_windows


#######
# if necessary, you can define additional functions which help your implementation,
#  and import proper libraries for those functions.
#######

class nn_convolutional_layer:


    def __init__(self, filter_width, filter_height, input_size, in_ch_size, num_filters, std=1e0):
        #  filter_width : 3, filter_height : 3, input_size : 32, in_ch_size = 3, num_filters : 8,  batch_size : 8 
        # initialization of weights
        self.W = np.random.normal(0, std / np.sqrt(in_ch_size * filter_width * filter_height / 2),
                                  (num_filters, in_ch_size, filter_width, filter_height))    # w = (8*3*3*3)
        self.b = 0.01 + np.zeros((1, num_filters, 1, 1))  # b = 1*8*1*1  
        self.input_size = input_size

        #######
        ## If necessary, you can define additional class variables here
        #######
        self.filter_width  = filter_width 
        self.filter_height = filter_height
        self.num_filters   = num_filters
        self.in_ch_size    = in_ch_size

    def update_weights(self, dW, db):
        self.W += dW
        self.b += db

    def get_weights(self):
        return self.W, self.b

    def set_weights(self, W, b):
        self.W = W
        self.b = b

    #######
    # Q1. Complete this method
    #######
    def forward(self, x):

        y = view_as_windows(x, (8,3,3,3))  # (x.shape[0], x.shape[1], self.filter_width, self.filter_height)  
        # convolution 연산 수행 (window : 8, 3, 3, 3)))  -> input 32 -> output (32-3+1=30) -> y (8(batch), 8(n_filter), 30, 30)
        # print('y : ', y.shape) # y_shape :  (1, 1, 30, 30, 8, 3, 3, 3)  

        # y, filter -> reshape 
        y = y.reshape (1, 1, 30, 30, 8, -1)  # (1,1,30,30,8,27)
        # print('y1 : ', y.shape)

        W = self.W.reshape(8, -1) # (8,27)
        # print('W : ', W.shape)

        rst = y @ W.T # (1,1,30,30,8,27)@(27,8)=(1,1,30,30,8,8)
        # print('rst1 : ', rst.shape)

        # squeeze (차원축소, axis 안주면 차원 중 사이즈가 1인 것을 찾아 스칼라값으로 바꿔 해당 차원을 제거한다)
        rst = rst.squeeze()   # (1,1,width_size, height_size, batch_size, num_filter) -> (width_size, height_size, batch_size, num_filter)
        # transpose(rst, (index 순서대로)) 원소의 index를 재배열    
        rst = np.transpose(rst,(2,3,0,1))  # (8,8,30,30) =  batch_size, num_filter_size, width_size, height_size 
        rst = rst + self.b   # wx + b 
        # print('rst2 : ', rst.shape)  # (8,8,30,30)

        return rst 

    #######
    # Q2. Complete this method
    # http://taewan.kim/post/cnn/
    # https://wiseodd.github.io/techblog/2016/07/16/convnet-conv-layer/
    #######
    # dLdx : q(dLdy,상수)를 padding -> w를 inverse -> convolution 
    # dLdw : x 와 dLdy를 convolution 
    # dLdb : np.sum(q) q의 모든 값을 더해준다 (모두 상수니) 

    def backprop(self, x, dLdy):

        # 1. dLdx : q(dLdy,상수)를 padding -> w를 inverse -> convolution 
        # 1-1) padding 처리  : https://dhhwang89.tistory.com/101
        # (batch_size, num_filter_size, width_size, height_size) 중 width_size, height_size 만 처리.
        pad = 2
        npad = ((0, 0), (0, 0), (pad, pad), (pad, pad))
        padding_dLdy = np.pad(dLdy, npad, 'constant', constant_values=(0))
        # print ('padding_dLdy : ', padding_dLdy.shape)

        dLdy_view = view_as_windows(padding_dLdy, (8,8,3,3)) # padding_dLdy.shape[0], padding_dLdy.shape[1], self.filter_width, self.filter_height
        # (batch size, num filter, width_size, height_size) (8,8,30,30)

        # 1-2) w 를 inverse처리 
        w_1 = np.flip(self.W, (2,3)) # reverse w. 0=batch size, 1=num filters는 그대로 두고 나머지만 (q) 바꾼다.
        dLdy_view = dLdy_view.reshape(dLdy_view.shape[0], dLdy_view.shape[1], dLdy_view.shape[2], dLdy_view.shape[3], dLdy_view.shape[4], -1)   
        w_1 = w_1.transpose(1,0,2,3)    # 순서변경 :  in_ch_size, num_filter, filter_width, filter_heigh
        w_1 = w_1.reshape(w_1.shape[0], -1) # 계산이 쉽게 flat 하게 변경

        # 1-3) convolution 진행 (dLdy @ W inverse)
        dLdx = dLdy_view @ w_1.T     
        dLdx = dLdx.squeeze()
        dLdx = dLdx.transpose(2,3,0,1)
        # print ('dLdx : ', dLdx.shape)


        # 2. dLdw : x 와 dLdy를 convolution 
        # 2-1) x : (batch size 8, input channel size 3 , in width 32, in height 32) 
        x_view = view_as_windows(x, (dLdy.shape[0],self.in_ch_size, dLdy.shape[2], dLdy.shape[3]))
        # in_ch_size, view_out_width, view_out_height, batch_size, filter(dLdy)_width, filter(dLdy)_height
        x_view = x_view.transpose (0, 1, 5, 2, 3, 4, 6, 7)
        x_view = x_view.reshape(x_view.shape[0], x_view.shape[1], x_view.shape[2], x_view.shape[3], x_view.shape[3], -1)

        # 2-2) dLdy :  (batch size 8, num filter 8, out width 30, out height 30)  
        dLdy_flat = dLdy.transpose(1,0,2,3)
        dLdy_flat = dLdy_flat.reshape(dLdy_flat.shape[0], -1)
        # print ('dLdy_flat : ', dLdy_flat.shape)

        # 2-3) convolution
        dLdW = x_view @ dLdy_flat.T
        # print('dLdW1 :', dLdW.shape)
        dLdW = dLdW.squeeze()
        dLdW = dLdW.transpose (3, 0, 1, 2)
        # (num_filters, in_ch_size, filter_width, filter_height)
        # print('dLdW2 shape :', dLdW.shape)
      
        # 3. dLdb : np.sum(q) q의 모든 값을 더해준다 (모두 상수) 
        dLdb = np.sum(dLdy, (0,2,3))  # filter의 갯수는 제외 후 더하기 axis=1
        dLdb = dLdb.reshape(self.b.shape)  # (1, 8, 1, 1)
        # print ('dLdb : ', dLdb.shape)

        return dLdx, dLdW, dLdb


class nn_max_pooling_layer:
    def __init__(self, stride, pool_size):
        self.stride = stride
        self.pool_size = pool_size
        #######
        ## If necessary, you can define additional class variables here
        #######

    #######
    # Q3. Complete this method
    #######
    def forward(self, x):
        y = view_as_windows(x, (x.shape[0], x.shape[1], self.pool_size, self.pool_size), self.stride)
        y = y.squeeze()
        y = y.reshape(y.shape[0], y.shape[1], y.shape[2], y.shape[3], -1)
        rst = y.max(axis=4)
        self.max_id = y.argmax(axis=4)
        self.max_id = self.max_id.transpose(2, 3, 0, 1)

        rst = rst.transpose(2, 3, 0, 1)
        return rst

    #######
    # Q4. Complete this method
    #######
    def backprop(self, x, dLdy):

        cal = np.zeros_like(x).astype(float)
        for i in np.arange(self.max_id.shape[0]):
            for j in np.arange(self.max_id.shape[1]):
                for k in np.arange(self.max_id.shape[2]):
                    for l in np.arange(self.max_id.shape[3]):
                        n = int(np.floor(self.max_id[i,j,k,l] / self.pool_size))
                        m = int (self.max_id[i,j,k,l] % self.pool_size)
                        cal[i,j, self.pool_size*k+n, self.pool_size*l+m] = dLdy[i,j,k,l]

        dLdx = cal
        return dLdx

    #######
    ## If necessary, you can define additional class methods here
    #######


# testing the implementation

# data sizes
batch_size = 8
input_size = 32
filter_width = 3
filter_height = filter_width
in_ch_size = 3
num_filters = 8

std = 1e0
dt = 1e-3

# number of test loops
num_test = 20

# error parameters
err_dLdb = 0
err_dLdx = 0
err_dLdW = 0
err_dLdx_pool = 0

for i in range(num_test):
    # create convolutional layer object
    cnv = nn_convolutional_layer(filter_width, filter_height, input_size, in_ch_size, num_filters, std)

    x = np.random.normal(0, 1, (batch_size, in_ch_size, input_size, input_size))
    delta = np.random.normal(0, 1, (batch_size, in_ch_size, input_size, input_size)) * dt

    # dLdx test
    print('dLdx test')
    y1 = cnv.forward(x)
    y2 = cnv.forward(x + delta)

    bp, _, _ = cnv.backprop(x, np.ones(y1.shape))

    exact_dx = np.sum(y2 - y1) / dt
    apprx_dx = np.sum(delta * bp) / dt
    print('exact change', exact_dx)
    print('apprx change', apprx_dx)

    err_dLdx += abs((apprx_dx - exact_dx) / exact_dx) / num_test * 100

    # dLdW test
    print('dLdW test')
    W, b = cnv.get_weights()
    dW = np.random.normal(0, 1, W.shape) * dt
    db = np.zeros(b.shape)

    z1 = cnv.forward(x)
    _, bpw, _ = cnv.backprop(x, np.ones(z1.shape))
    cnv.update_weights(dW, db)
    z2 = cnv.forward(x)

    exact_dW = np.sum(z2 - z1) / dt
    apprx_dW = np.sum(dW * bpw) / dt
    print('exact change', exact_dW)
    print('apprx change', apprx_dW)

    err_dLdW += abs((apprx_dW - exact_dW) / exact_dW) / num_test * 100

    # dLdb test
    print('dLdb test')

    W, b = cnv.get_weights()

    dW = np.zeros(W.shape)
    db = np.random.normal(0, 1, b.shape) * dt

    z1 = cnv.forward(x)

    V = np.random.normal(0, 1, z1.shape)

    _, _, bpb = cnv.backprop(x, V)

    cnv.update_weights(dW, db)
    z2 = cnv.forward(x)

    exact_db = np.sum(V * (z2 - z1) / dt)
    apprx_db = np.sum(db * bpb) / dt

    print('exact change', exact_db)
    print('apprx change', apprx_db)
    err_dLdb += abs((apprx_db - exact_db) / exact_db) / num_test * 100

    # max pooling test
    # parameters for max pooling
    stride = 2
    pool_size = 2

    mpl = nn_max_pooling_layer(stride=stride, pool_size=pool_size)

    x = np.arange(batch_size * in_ch_size * input_size * input_size).reshape(
        (batch_size, in_ch_size, input_size, input_size)) + 1
    delta = np.random.normal(0, 1, (batch_size, in_ch_size, input_size, input_size)) * dt

    print('dLdx test for pooling')
    y1 = mpl.forward(x) #(8, 3, 16, 16)
    dLdy = np.random.normal(0, 10, y1.shape)
    bpm = mpl.backprop(x, dLdy)

    y2 = mpl.forward(x + delta)

    exact_dx_pool = np.sum(dLdy * (y2 - y1)) / dt
    apprx_dx_pool = np.sum(delta * bpm) / dt
    print('exact change', exact_dx_pool)
    print('apprx change', apprx_dx_pool)

    err_dLdx_pool += abs((apprx_dx_pool - exact_dx_pool) / exact_dx_pool) / num_test * 100

# reporting accuracy results.
print('accuracy results')
print('conv layer dLdx', 100 - err_dLdx, '%')
print('conv layer dLdW', 100 - err_dLdW, '%')
print('conv layer dLdb', 100 - err_dLdb, '%')
print('maxpool layer dLdx', 100 - err_dLdx_pool, '%')

dLdx test
exact change 461.11104977408536
apprx change 461.1110497740586
dLdW test
exact change 3565.04967421296
apprx change 3565.0496742129926
dLdb test
exact change 39.97687933679555
apprx change 39.97687933678098
dLdx test for pooling
exact change 148.08934442882978
apprx change 148.0893445543903
dLdx test
exact change -219.11107699563462
apprx change -219.11107699569922
dLdW test
exact change 54.68333767686775
apprx change 54.68333767689604
dLdb test
exact change -217.5668117061187
apprx change -217.56681170612023
dLdx test for pooling
exact change 144.07517017340209
apprx change 144.07517064192425
dLdx test
exact change 457.43233603645524
apprx change 457.43233603657717
dLdW test
exact change -258.97503877123796
apprx change -258.97503877113945
dLdb test
exact change 376.8919130567085
apprx change 376.8919130566851
dLdx test for pooling
exact change -726.9507508633988
apprx change -726.950750951918
dLdx test
exact change -148.78660204799695
apprx change -148.7866020478754
dLdW te